# LU decomposition with complete pivoting

$$
PAQ^T = LU
$$

where 
- $P=P_{n-1} \cdots P_1$;   $P_k$ is row interchange matrices at $k$-th step
- $Q=Q_{n-1} \cdots Q_1$; $Q_k$ is column interchange matrices at $k$-th step
- $L$: unit lower triangular
- $U$: uppder triangular

Q may be defined as $Q := Q_1 \cdots Q_{n-1}$. Then, $PAQ=LU$.



In [1]:
import numpy as np 


def lu_decomposition_complete_pivoting(A=None, n=5, debug=False):
    if A is None:
        A_orig = np.random.randint(-9, 9, size=(n, n)).astype(np.float64)
        A = A_orig.copy()
    else:
        A_orig = A
        A = A_orig.copy() # do not modify original A
        n = len(A)

    # Ensure the matrix is square
    if A.shape[0] != A.shape[1]:
        raise ValueError("Matrix A must be square for LU decomposition.")

    # Initialize swap memos
    row_swaps = []
    col_swaps = []

    if debug:
        print("\nOriginal A:")
        print(A_orig)
        print("\nPerform LU decomposition with complete pivoting")
    
    for k in range(n-1):
        if debug:
            print("\n***********************************************")
            print(f"{k}th step")
            
        # Find pivot position
        pos = np.unravel_index(np.argmax(abs(A[k:, k:])), A[k:, k:].shape)
        pivot_row = k + pos[0]
        pivot_col = k + pos[1]
    
        # Record swaps in the memo
        row_swaps.append((k, pivot_row))
        col_swaps.append((k, pivot_col))
        
        # Apply the swaps to matrix A
        if k != pivot_row:
            A[[k, pivot_row], :] = A[[pivot_row, k], :]
        if k != pivot_col:
            A[:, [k, pivot_col]] = A[:, [pivot_col, k]]
    
        if debug:
            print(f"\nPivot position: ({pivot_row}, {pivot_col})")
            print("\nMatrix A after swaps:")
            print(A)
    
        # Update matrix A (Gaussian elimination step)
        if A[k, k] != 0:
            A[k+1:, k] /= A[k, k]
            A[k+1:, k+1:] -= np.outer(A[k+1:, k], A[k, k+1:])
            
        if debug:
            print("\nOuter product Updated A:")
            print(A)
    
    # Construct permutation matrices using the swap memo
    P = np.eye(n)
    Q = np.eye(n)
    
    # Apply row swaps to P (row-based)
    for r1, r2 in row_swaps:
        P[[r1, r2]] = P[[r2, r1]]
    
    # Apply column swaps to Q (column-based)
    for c1, c2 in col_swaps:
        Q[[c1, c2]] = Q[[c2, c1]]
    
    # Construct L and U
    L = np.eye(n)
    U = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i > j:
                L[i, j] = A[i, j]
            else:
                U[i, j] = A[i, j]
    
    # Final results
    if debug:
        print("*********************************************")
        print("Final Result")
        print("\nP (Row Permutation Matrix):")
        print(P)
        print("\nQ (Column Permutation Matrix):")
        print(Q)
        print("\nL (Lower Triangular Matrix):")
        print(L)
        print("\nU (Upper Triangular Matrix):")
        print(U)

    # Reconstructed matrix and error check
    reconstructed_A = P.T @ L @ U @ Q
    error = np.linalg.norm(reconstructed_A - A_orig)
    if debug:
        print("\nReconstructed Matrix P.T @ L @ U @ Q:")
        print(reconstructed_A)
        print("\nOriginal A:")
        print(A_orig)
        print(f"\nReconstruction Error: {error:.2e}")
    
    # Checking the determinant of the original matrix
    return P, Q, L, U, error


# Example
A = np.array([
    [0, 2, 3],
    [3, 2, 4] ,
    [4, 5, 4],
]).astype(np.float64)

P, Q, L, U, error = lu_decomposition_complete_pivoting(A, debug=True)


# 100 tests
num_tests = 100
errors = []
print("\n****************************************")
print("Testing 100 times")
for i in range(num_tests):
    P, Q, L, U, error = lu_decomposition_complete_pivoting(debug=False)
    errors.append(error)

print("mean error:", np.mean(errors))


Original A:
[[0. 2. 3.]
 [3. 2. 4.]
 [4. 5. 4.]]

Perform LU decomposition with complete pivoting

***********************************************
0th step

Pivot position: (2, 1)

Matrix A after swaps:
[[5. 4. 4.]
 [2. 3. 4.]
 [2. 0. 3.]]

Outer product Updated A:
[[ 5.   4.   4. ]
 [ 0.4  1.4  2.4]
 [ 0.4 -1.6  1.4]]

***********************************************
1th step

Pivot position: (1, 2)

Matrix A after swaps:
[[ 5.   4.   4. ]
 [ 0.4  2.4  1.4]
 [ 0.4  1.4 -1.6]]

Outer product Updated A:
[[ 5.          4.          4.        ]
 [ 0.4         2.4         1.4       ]
 [ 0.4         0.58333333 -2.41666667]]
*********************************************
Final Result

P (Row Permutation Matrix):
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]

Q (Column Permutation Matrix):
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]

L (Lower Triangular Matrix):
[[1.         0.         0.        ]
 [0.4        1.         0.        ]
 [0.4        0.58333333 1.        ]]

U (Upper Triangular Matrix):
[[ 5.      

## More example: Compute determinant 

maybe not efficient 

In [6]:
import numpy as np

# Example
n = 2
A = np.random.randint(11, 99, size=(n,n)).astype(np.float64)

P, Q, L, U, error = lu_decomposition_complete_pivoting(A, debug=True)

print("\nA")
print(A)
print("\ndet(A) computed by numpy: ", np.linalg.det(A))
print("\ndet(A) computed by complete pivoting:", np.prod(np.diag(U)))
print("\ndet(A) computed by known formula", A[0,0]*A[1,1]-A[0,1]*A[1,0])


Original A:
[[29. 49.]
 [96. 34.]]

Perform LU decomposition with complete pivoting

***********************************************
0th step

Pivot position: (1, 0)

Matrix A after swaps:
[[96. 34.]
 [29. 49.]]

Outer product Updated A:
[[96.         34.        ]
 [ 0.30208333 38.72916667]]
*********************************************
Final Result

P (Row Permutation Matrix):
[[0. 1.]
 [1. 0.]]

Q (Column Permutation Matrix):
[[1. 0.]
 [0. 1.]]

L (Lower Triangular Matrix):
[[1.         0.        ]
 [0.30208333 1.        ]]

U (Upper Triangular Matrix):
[[96.         34.        ]
 [ 0.         38.72916667]]

Reconstructed Matrix P.T @ L @ U @ Q:
[[29. 49.]
 [96. 34.]]

Original A:
[[29. 49.]
 [96. 34.]]

Reconstruction Error: 0.00e+00

A
[[29. 49.]
 [96. 34.]]

det(A) computed by numpy:  -3717.9999999999977

det(A) computed by complete pivoting: 3718.0000000000005

det(A) computed by known formula -3718.0


## Matrix Rank revelation

Find the Rank of rank-deficient matrix A using complete pivoting 

- $A:= U V^T$: rank-k matrix
- $U,V \in \mathbf{R}^{n \times k}$ 

In [3]:
import numpy as np

n = 4
k = 2
U = np.random.randint(11,99, size=(n,k)).astype(np.float64)
V = np.random.randint(11,99, size=(n,k)).astype(np.float64)
A = U@V.T # rank2 matrix
P, Q, L, U, error = lu_decomposition_complete_pivoting(A.copy(), debug=False)
reconstructed_A = P.T @ L @ U @ Q

print("\nA:")
print(A)
print("\nU (rounded to 8 decimal places): ")
print(np.round(U, decimals=8))
print("\nReconstructed A=P.T @ L @ U @ Q.T:")
print(reconstructed_A)


A:
[[ 4574.  2002.  3712.  2608.]
 [13740.  6185. 11310.  7640.]
 [ 4232.  1300.  2920.  3040.]
 [ 4288.  1557.  3182.  2808.]]

U (rounded to 8 decimal places): 
[[13740.          7640.         11310.          6185.        ]
 [    0.           686.83551674  -563.54585153  -605.01601164]
 [    0.             0.            -0.            -0.        ]
 [    0.             0.             0.             0.        ]]

Reconstructed A=P.T @ L @ U @ Q.T:
[[ 4574.  2002.  3712.  2608.]
 [13740.  6185. 11310.  7640.]
 [ 4232.  1300.  2920.  3040.]
 [ 4288.  1557.  3182.  2808.]]


In [5]:
import numpy as np

n = 6
k = 4
U = np.random.randint(0,3, size=(n,k)).astype(np.float64)
V = np.random.randint(0,3, size=(n,k)).astype(np.float64)
A = U@V.T # rank-k matrix

P, Q, L, U, error = lu_decomposition_complete_pivoting(A.copy(), debug=False)
reconstructed_A = P.T @ L @ U @ Q

print("\nA:")
print(A)
print("\nU (rounded to 8 decimal places): ")
print(np.round(U, decimals=8))
print("\nReconstructed A (rounded to 16 decimal places)::")
print(np.round(reconstructed_A, decimals=8))


A:
[[ 0.  2.  0.  4.  0.  2.]
 [ 2.  7.  3.  4.  1.  8.]
 [ 4.  6.  2.  5.  2.  8.]
 [ 2.  8.  3.  5.  1.  9.]
 [ 0.  2.  1.  1.  0.  2.]
 [ 4.  9.  3.  8.  2. 11.]]

U (rounded to 8 decimal places): 
[[11.          8.          4.          3.          9.          2.        ]
 [ 0.          2.54545455 -0.72727273 -0.54545455  0.36363636 -0.36363636]
 [ 0.          0.         -1.71428571  0.21428571  0.85714286 -0.85714286]
 [ 0.          0.          0.          0.25        0.          0.        ]
 [ 0.          0.          0.          0.         -0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]]

Reconstructed A (rounded to 16 decimal places)::
[[ 0.  2.  0.  4.  0.  2.]
 [ 2.  7.  3.  4.  1.  8.]
 [ 4.  6.  2.  5.  2.  8.]
 [ 2.  8.  3.  5.  1.  9.]
 [-0.  2.  1.  1. -0.  2.]
 [ 4.  9.  3.  8.  2. 11.]]


## References:


* matrix computations 4th edition
* https://en.wikipedia.org/wiki/LU_decomposition
